# Environment finder

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, widgets
import ase
from ase.io import read
from ase.neighborlist import NeighborList
from ase.visualize.plot import plot_atoms
from itertools import permutations

## Select structure

In [3]:
def plotConf(anglex,angley,anglez):
    global conf, atom_types
    plt.rcParams['figure.figsize'] = [15, 7]
    fig, ax = plt.subplots()
    anglestr = str(anglex) + "x," + str(angley) + "y," + str(anglez) + "z"
    plot_atoms(conf, ax, radii=0.3, rotation=(anglestr))
    plt.show()

def f(x):
    global conf, atom_types
    filename=x
    conf = read(filename)
    atom_types = np.unique(conf.get_chemical_symbols())
    w = interactive(plotConf, anglex=widgets.IntSlider(description='Angle x:',min=-90,max=90,step=5,value=0), angley=widgets.IntSlider(description='Angle y:',min=-90,max=90,step=5,value=0), anglez=widgets.IntSlider(description='Angle z:',min=-90,max=90,step=5,value=0))
    display(w)

  
interact(f, x=[('Urea', 'Examples/urea2.pdb'), ('Alpha Ga', 'Examples/a-ga.xyz')]);

interactive(children=(Dropdown(description='x', options=(('Urea', 'Examples/urea2.pdb'), ('Alpha Ga', 'Example…

In [4]:
# Rotations if needed
#conf.rotate(125, (-1, -1, 0))
#conf.rotate(15, (0, 0, 1))
#conf.rotate(180, (1, 0, 0))

## Get template

In [ ]:
#@title {display-mode: "form"}

fractional=False
# Structure for templates
class Environments:
    def __init__(self):
        self.myindex = -1
        self.indeces = np.array([],dtype=np.int)
        self.delta = np.array([])
        self.distance =  np.array([])

def calculateVectors(lista,listb,cutoff,tolerance):
    global Template, unique_environments, conf, atom_types
    Template = np.ndarray((0,),dtype=np.object)
    # Find all environments
    # Loop over input particles of type atom_type_1:
    myindeces=lista #np.argwhere(chemical_symbols == atom_type_1)
    nl = NeighborList((cutoff/2.0)*np.ones(conf.get_number_of_atoms()),skin=0.1,bothways=True)
    nl.update(conf)
    mat = conf.get_cell()
    #print(mat)
    for index in myindeces:
        neighbors, offsets = nl.get_neighbors(index)
        # Iterate over the neighbors of the current particle:
        Environment = Environments()
        Environment.myindex = index
        Environment.delta.shape = (0,3)
        for neigh, offset in zip(neighbors, offsets):
            #if (chemical_symbols[neigh]==atom_type_2): # Only if neighbor is in listb
            if (neigh in listb): # Only if neighbor is in listb
                delta = conf.positions[neigh] + np.dot(offset, mat) - conf.positions[index]
                distance=np.linalg.norm(delta)
                if (distance>1.e-10):
                    if (not fractional):
                        Environment.delta = np.append(Environment.delta,np.array([[delta[0], delta[1], delta[2]]])*0.1,axis=0)
                    else:
                        Environment.delta = np.append(Environment.delta,np.array([[delta[0]/mat[0][0], delta[1]/mat[1][1], delta[2]/mat[2][2]]]),axis=0)
                    Environment.distance = np.append(Environment.distance,distance)
                    Environment.indeces = np.append(Environment.indeces,neigh)
        Template = np.append(Template,Environment)
    # Compare environments
    num_of_templates=Template.shape[0]
    print("num_of_templates",num_of_templates)
    flag_unique=np.ones(num_of_templates)
    for i in range(num_of_templates):
        if (Template[i].indeces.shape[0]==0):
            flag_unique[i]=0
            break
        if (flag_unique[i]==1):
            for j in range(i+1,num_of_templates):
                perms = set(permutations(np.arange(Template[j].indeces.shape[0])))
                print(perms)
                for p in perms:
                    print(np.array(p).shape)
                    print(xyz)
                    if (Template[j].indeces.shape[0]==0):
                        flag_unique[j]=0
                        break
                    condition1=not(Template[i].indeces.shape[0]==Template[j].indeces.shape[0])
                    #print(Template[i].delta.shape,Template[j].delta.shape)
                    condition2=(np.count_nonzero(np.isclose(Template[i].delta,Template[j].delta[np.array(p),:],atol=tolerance))==(Template[i].indeces.shape[0]*3))
                    if ( condition1 or condition2):
                        flag_unique[j]=0
                        break
    # Print C++ syntax or plumed input syntax
    unique_environments = np.array([],dtype=np.int)
    counter = 0
    for i in range(num_of_templates):
        if (flag_unique[i]==1):
            for j in range(Template[i].indeces.shape[0]):
                print("VECTOR" + str(counter+1) + "_" + str(j+1) + "=" + str(Template[i].delta[j,0]) + ","  + str(Template[i].delta[j,1]) + "," + str(Template[i].delta[j,2]))
                #print("DISTANCE" + str(counter+1) + "_" + str(j+1) + "=" + str(Template[i].distance[j]))
            counter += 1
            unique_environments = np.append(unique_environments,i)
      


def calculateVectorsType(atom_type_1,atom_type_2,cutoff,tolerance):
    global conf, atom_types
    chemical_symbols = np.asarray(conf.get_chemical_symbols())
    lista=np.argwhere(chemical_symbols == atom_type_1).flatten()
    listb=np.argwhere(chemical_symbols == atom_type_2).flatten()
    calculateVectors(lista,listb,cutoff,tolerance)
    
def calculateVectorsString(listastring,listbstring,cutoff,tolerance):
    global conf, atom_types
    lista=np.fromstring(listastring, dtype=int, sep=',')-1
    listb=np.fromstring(listbstring, dtype=int, sep=',')-1
    calculateVectors(lista,listb,cutoff,tolerance)
    
def calculateVectorsMinMaxStride(mina,maxa,stridea,minb,maxb,strideb,cutoff,tolerance):
    global conf, atom_types
    lista=np.arange(int(mina),int(maxa)+1,int(stridea))-1
    listb=np.arange(int(minb),int(maxb)+1,int(strideb))-1
    print(lista,listb)
    calculateVectors(lista,listb,cutoff,tolerance)
    
#wvec

def toggleTypeAndIndex(value):
    global conf, atom_types
    if (value=='Type'):
        # Call function with widgets
        wvec = interactive(calculateVectorsType, 
            atom_type_1 = widgets.Dropdown(options=atom_types, value=atom_types[0], description='Atom type 1:'),
            atom_type_2 = widgets.Dropdown(options=atom_types, value=atom_types[0], description='Atom type 2:'),
            cutoff =  widgets.FloatText(value=5,description='Cutoff:',disabled=False),
            tolerance = widgets.FloatText(value=0.02,description='Tolerance:',disabled=False) 
        )
        display(wvec)
    elif (value=='String'):
        # Call function with widgets
        wvec = interactive(calculateVectorsString, 
                   listastring = widgets.Text(value="1,2,3", description='List A:',placeholder='Type something',disabled=False),
                   listbstring = widgets.Text(value="1,2", description='List B:',placeholder='Type something',disabled=False),
                   cutoff =  widgets.FloatText(value=3,description='Cutoff:',disabled=False),
                   tolerance = widgets.FloatText(value=0.02,description='Tolerance:',disabled=False) 
        )
        display(wvec)
    elif (value=='Step'):
        # Call function with widgets
        wvec = interactive(calculateVectorsMinMaxStride,
                   mina = widgets.Text(value="1", description='Min A:',placeholder='Type something',disabled=False),
                   maxa = widgets.Text(value="2", description='Max A:',placeholder='Type something',disabled=False),
                   stridea = widgets.Text(value="1", description='Stride A:',placeholder='Type something',disabled=False),
                   minb = widgets.Text(value="1", description='Min B:',placeholder='Type something',disabled=False),
                   maxb = widgets.Text(value="2", description='Max B:',placeholder='Type something',disabled=False),
                   strideb = widgets.Text(value="1", description='Stride B:',placeholder='Type something',disabled=False),
                   cutoff =  widgets.FloatText(value=5,description='Cutoff:',disabled=False),
                   tolerance = widgets.FloatText(value=0.02,description='Tolerance:',disabled=False) 
        )
        display(wvec)
    else:
        print("Error")
         
wtoggle = interactive(toggleTypeAndIndex, value=widgets.ToggleButtons(options=['Type','String','Step'], description='Choose:', disabled=False) )
         
display(wtoggle)

interactive(children=(ToggleButtons(description='Choose:', options=('Type', 'String', 'Step'), value='Type'), …

## Plot environments

In [10]:
#@title {display-mode: "form"}

def plotEnv(number,anglex,angley,anglez):
  env_number=unique_environments[int(number-1)]
  print(env_number)
  fig, ax = plt.subplots()
  anglestr = str(anglex) + "x," + str(angley) + "y," + str(anglez) + "z"
  #print( np.append(Template[env_number].indeces.astype(int),Template[env_number].myindex) )
  #print(Template[env_number].indeces.astype(int))
  env_atom_types = np.asarray(conf.get_chemical_symbols())[Template[env_number].indeces.astype(int)] #,np.array('C')] #Template[env_number].myindex)]
  env_atom_types = np.append(env_atom_types,np.array('Si'))
  #print(env_atom_types)
  env_positions = np.vstack((Template[env_number].delta*10,np.array([0,0,0]) ) ) 
  #print(env_positions)
  env = ase.Atoms(env_atom_types,env_positions)
  plot_atoms(env, ax, radii=0.3, rotation=(anglestr))
  plt.show()

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
w = interactive(plotEnv, number=widgets.IntSlider(description='Environment:',min=1,max=unique_environments.shape[0],step=1,value=0), anglex=widgets.IntSlider(description='Angle x:',min=-90,max=90,step=5,value=0), angley=widgets.IntSlider(description='Angle y:',min=-90,max=90,step=5,value=0), anglez=widgets.IntSlider(description='Angle z:',min=-90,max=90,step=5,value=0))
display(w)

interactive(children=(IntSlider(value=1, description='Environment:', max=6, min=1), IntSlider(value=0, descrip…